# Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LinearRegression

import numpy as np
from sklearn.model_selection import KFold

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import os
import math

In [2]:
# Mouting with google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Config path and variable

In [3]:
# Path to data folder
data_folder = '/content/drive/MyDrive/STUDYING/HK2 2020-2021/Modern Problems in Technology/'

# Choose one of 2 versions below:
# Uncomment below line to choose scaled data version
# data_version = 'data_processed_withscale.csv'|

# Uncomment below line to choose non-scaled data version
data_version = 'data_processed_withscale_full_features.csv'

## Reading data version

In [4]:
data = pd.read_csv(os.path.join(data_folder, data_version))

data


,time,lat,lon,NO2,name,pblh,press,rh,tmp,wspd,dpt_00,dpt_06,dpt_12,dpt_18,ndvi,pplt_dens,road_dens,cluster
0,11,0.945033,0.181002,0.000000,NVC,0.295189,0.992355,0.503963,0.006589,0.267935,0.180598,0.276074,0.228643,0.194980,0.134988,0.062616,0.204322,3
1,12,0.945033,0.181002,0.000000,NVC,0.431093,0.970994,0.491827,0.045554,0.192998,0.062569,0.247647,0.354218,0.289970,0.000000,0.062616,0.204322,3
2,13,0.945033,0.181002,41.310400,NVC,0.241775,0.915679,0.543707,0.081644,0.150712,0.172614,0.324583,0.358503,0.342192,0.000000,0.062616,0.204322,3
3,14,0.945033,0.181002,39.379204,NVC,0.137111,0.854518,0.683671,0.178665,0.137490,0.286272,0.454138,0.499029,0.491546,0.000000,0.062616,0.204322,3
4,15,0.945033,0.181002,39.901879,NVC,0.210562,0.830009,0.642043,0.261890,0.161539,0.413801,0.517287,0.572602,0.437929,0.000000,0.062616,0.204322,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,1227,0.939449,0.156999,18.995833,ThanhCong,0.346466,0.829110,0.160080,0.227323,0.197763,0.232337,0.278541,0.204074,0.253429,0.000000,1.000000,0.793592,0
5456,1228,0.939449,0.156999,23.241667,ThanhCong,0.301871,0.758730,0.249164,0.333443,0.112616,0.445943,0.492757,0.243412,0.343498,0.000000,1.000000,0.793592,0
5457,1229,0.939449,0.156999,16.816667,ThanhCong,0.241945,0.703640,0.501023,0.452160,0.047686,0.597599,0.629433,0.505862,0.644568,0.000000,1.000000,0.793592,0
5458,1230,0.939449,0.156999,21.595833,ThanhCong,0.357267,0.767949,0.479585,0.471869,0.227892,0.560463,0.559629,0.590930,0.643036,0.000000,1.000000,0.793592,0


In [5]:
# data = data[data.cluster != 2]
# Unique cluster value
data['cluster'].unique()

array([3, 2, 1, 0])

## Remove Invalid NO2 values

In [6]:
# Run this if you want to remove invalid no2 values

data = data[(data.NO2 > 0) & (data.NO2 < 100)]
data[data.NO2 > 100]

,time,lat,lon,NO2,name,pblh,press,rh,tmp,wspd,dpt_00,dpt_06,dpt_12,dpt_18,ndvi,pplt_dens,road_dens,cluster


## Drop useless columns

In [ ]:
def drop_cols(df):
  drops = ['lat', 'lon', 'cluster', 'dpt_00', 'dpt_06', 'dpt_12', 'dpt_18']
  for col in drops:
    if col in df:
      df.drop(col, axis='columns', inplace=True)

# drop_cols(data)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## Convert to numpy for feeding model

In [7]:
X = data.drop(['NO2', 'name'], axis = 'columns')
X_arr = X.to_numpy()

y = data['NO2']
y_arr = y.to_numpy()

print(X.shape, y.shape)

(4679, 16) (4679,)


In [ ]:
# X_train = X.drop(['dpt_00', 'dpt_06', 'dpt_12', 'dpt_18'], axis='columns')

In [8]:
# # X_train = X_train.drop(['lat','lon','cluster'], axis='columns')
# X_train.shape

In [9]:
# List of RMSE on all folds
# rmses_rf = []
# mapes_rf = []
# r2s_rf = []

# rmses_lr = []
# mapes_lr = []
# r2s_lr = []

# Depth of Random Forest
depth = 100
# Number of split
split_number = 10

map_depth_rmse = {}
map_depth_r2 = {}
map_depth_mape = {}

# How many number want to round with loss (1.424 if precision =3 for example)
precision = 5

def get_key(dict, val):
  for key, value in dict.items():
        if val == value:
            return key

  return "key doesn't exist"

def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

# KFold for each depth
for d in range(1, depth):
  kf = KFold(n_splits=split_number, random_state=None, shuffle=True)
  kf.get_n_splits(X)

  rmses_rf = []
  mapes_rf = []
  r2s_rf = []

  # rmses_lr = []
  # mapes_lr = []
  # r2s_lr = []

  for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_arr[train_index], X_arr[test_index]
    y_train, y_test = y_arr[train_index], y_arr[test_index]
    # print(len(train_index), len(test_index))

    # print('Fold {}:'.format(i))
    rf = RandomForestRegressor(n_estimators = d, random_state = 20)
    rf.fit(X_train, y_train)
    rf_predicts = rf.predict(X_test)
    # Calculate RMSE
    rmse_rf = math.sqrt(mean_squared_error(y_test, rf_predicts))
    # Calculate R2 Score
    r2_rf = r2_score(y_test, rf_predicts)
    # Calculate mean absolute error (MAE NOT MAPE)
    # mape_rf = mean_absolute_error(y_test, rf_predicts)
    mape_rf = MAPE(y_test, rf_predicts)

    # With each depth, calculate and save losses of all fold
    rmses_rf.append(rmse_rf)
    r2s_rf.append(r2_rf)
    mapes_rf.append(mape_rf)

  # After each depth, save average loss to find best depth
  average_rmse_rf = round(np.average(rmses_rf), precision)
  average_r2_rf = round(np.average(r2s_rf), precision)
  average_mape_rf = round(np.average(mapes_rf), precision)

  map_depth_rmse[d] = average_rmse_rf
  map_depth_r2[d] = average_r2_rf
  map_depth_mape[d] = average_mape_rf
  
  print('Depth =', d)
  print('Average RMSE on all folds using Random Forest:', average_rmse_rf)
  print('Average R2 on all folds using Random Forest:', average_r2_rf)
  print('Average MAPE on all folds using Random Forest: {}'.format(average_mape_rf))
  # print('Average MAPE on all folds using Random Forest:', np.average(mapes_rf))
  print()

min_rmse = min(map_depth_rmse.values())
max_r2 = max(map_depth_r2.values())
min_mape = min(map_depth_mape.values())

print('Best depth for RMSE is {} with RMSE = {}'.format(get_key(map_depth_rmse, min_rmse), min_rmse))
print('Best depth for R2 is {} with R2 = {}'.format(get_key(map_depth_r2, max_r2), max_r2))
print('Best depth for R2 is {} with MAPE = {}'.format(get_key(map_depth_mape, min_mape), min_mape))

  # # Linear Regression
  # lr = LinearRegression()
  # lr.fit(X_train, y_train)
  # lr_predicts = lr.predict(X_test)
  # # Calculate RMSE
  # rmse_lr = math.sqrt(mean_squared_error(y_test, lr_predicts))
  # # Calculate R2 Score
  # r2_lr = r2_score(y_test, lr_predicts)
  # # Calculate mean absolute error (MAE NOT MAPE)
  # mape_lr = mean_absolute_error(y_test, lr_predicts)

  # if rmse_lr < min_rmse_lr:
  #   min_rmse_lr = rmse_lr
  # if r2_lr < min_r2_lr:
  #   min_r2_lr = r2_lr
  # if mape_lr < min_mape_lr:
  #   min_mape_lr = mape_lr

  # # a={'true': y_test, 'predict':predicts}
  # # display(pd.DataFrame(a))

  # rmses_lr.append(min_rmse_lr)
  # r2s_lr.append(min_r2_lr)
  # mapes_lr.append(min_mape_lr)

  # print('Min RMSE for Linear Regression:'.format(i), min_rmse_lr)
  # print('Min R2 for Linear Regression:'.format(i), min_r2_lr)
  # print('Min MAPE for Linear Regression:'.format(i), min_mape_lr)
  # print()
  # print()

  # i += 1

# print()
# print('Average RMSE on all folds using Linear Regression:', np.average(rmses_lr))
# print('Average R2 on all folds using Linear Regression:', np.average(r2s_lr))
# print('Average MAPE on all folds using Linear Regression:', np.average(mapes_lr))


Depth = 1
Average RMSE on all folds using Random Forest: 12.37413
Average R2 on all folds using Random Forest: 0.32579
Average MAPE on all folds using Random Forest: 44.92978

Depth = 2
Average RMSE on all folds using Random Forest: 10.67022
Average R2 on all folds using Random Forest: 0.49704
Average MAPE on all folds using Random Forest: 40.47478

Depth = 3
Average RMSE on all folds using Random Forest: 9.89279
Average R2 on all folds using Random Forest: 0.56366
Average MAPE on all folds using Random Forest: 44.20625

Depth = 4
Average RMSE on all folds using Random Forest: 9.44678
Average R2 on all folds using Random Forest: 0.60244
Average MAPE on all folds using Random Forest: 35.54301

Depth = 5
Average RMSE on all folds using Random Forest: 9.31089
Average R2 on all folds using Random Forest: 0.61715
Average MAPE on all folds using Random Forest: 40.48426

Depth = 6
Average RMSE on all folds using Random Forest: 9.15733
Average R2 on all folds using Random Forest: 0.63016
Avera